## Classiq's YQuantum 2024 Sparse State-Preparation Challenge

Welcome to the Sparse State-Preparation Challenge, presented by Classiq at YQuantum 2024. This challenge is designed for quantum computing enthusiasts of all levels, aiming to enhance your understanding and spur innovation in sparse state-preparation.

### Challenge Overview
- **Objective**: Develop quantum circuits that efficiently prepare sparse quantum states. These states are characterized by a small number of non-zero amplitudes, reducing quantum resource requirements and enhancing practical execution on near-term quantum hardware.

### Background
- **Importance of State Preparation**: State preparation is a crucial first step for many quantum algorithms. It involves setting a quantum state to represent a specific vector of probabilities or functions. The challenge focuses on sparse states, which are pivotal in applications like quantum linear algebra and quantum machine learning.

### Problem Statement
- **Sparse State-Preparation**: Your task is to create circuits that load a quantum state with probabilities corresponding to a given sparse vector. This challenge highlights the nuances of designing circuits that are both efficient and effective for today's quantum computing limitations.

### Resources and Guidance
- **Classiq's Tools**: Utilize Classiq’s platform, QMOD language, and SDK, which provide robust tools for designing and testing quantum algorithms. Participants are encouraged to base their implementations on "An Efficient Algorithm for Sparse Quantum State Preparation" by Niels Gleinig and Torsten Hoefler.

### Goals and Expectations
- **Output Goal**: Efficiently prepare a quantum state reflecting specified probabilities, ensuring high fidelity and resource efficiency. 
- **Example**: For input `{‘00000001’: 0.25, ‘00010001’: 0.5, ‘11001000’: 0.25}`, the output quantum state should correspond to these probabilities.


In [1]:
import classiq
classiq.authenticate()

/home/rdang02/.local/lib/python3.8/site-packages/classiq/_internals/authentication/token_manager.py:96: UserWarning: Device is already registered.
Generating a new refresh token should only be done if the current refresh token is compromised.
To do so, set the overwrite parameter to true
  warnings.warn(


In [2]:
from classiq import *
import numpy as np

## Classical Functions
This section includes utility functions for classical pre-processing in quantum algorithms:

- `dict_to_3d_array(sparse_states)`: Converts a dictionary of sparse states into a 3D array format.
- `custom_filter(func, iterable)`: Filters items in an iterable based on a function.
- `unequal_sets(t, n)`: Determines the best qubit to split a set `t` into subsets with a significant size difference.
- `process_subsets(t, n, dif_qubits, dif_values)`: Processes subsets to determine difference qubits and values.
- `toggle_operations(index, n, x_x, gates, x_qubits, s)`: Toggles operations based on the index condition.
- `conditional_toggle(gates, x_qubits, n, dif, b, s)`: Conditionally toggles based on a difference.
- `calc_alpha_beta(x_1, x_2)`: Calculates and adjusts alpha and beta values.

In [3]:
def dict_to_3d_array(sparse_states):
    array_3d = []
    
    for key, value in sparse_states.items():
        point = [int(digit) for digit in key]
        array_3d.append([point, value])
    
    return array_3d

def custom_filter(func, iterable):
    return [item for item in iterable if func(item)]

def unequal_sets(t, n): 

    best_qubit = None
    T_0=[]
    T_1=[]
    current_difference = float('-inf')

    for b in range(n):    
        #Filter list based on boolean condition
        T_0 = custom_filter(lambda x, b=b: x[0][b] == 0, t)
        T_1 = custom_filter(lambda x, b=b: x[0][b] == 1, t)

        # Check if both sets are non-empty
        if len(T_0) != 0 and len(T_1) != 0:
            difference = abs(len(T_0) - len(T_1))
            #If new max difference
            if difference > current_difference:
                current_difference = difference
                best_qubit = b
                t_0 = T_0
                t_1 = T_1
    
    return best_qubit,t_0, t_1

def process_subsets(t, n, dif_qubits, dif_values):
    while len(t) > 1:
        b, T_0, T_1 = unequal_sets(t, n)
        dif_qubits.append(b)
        if len(T_0) < len(T_1):
            t = T_0
            dif_values.append(0)
        else:
            t = T_1
            dif_values.append(1)
    return dif_qubits, dif_values, t

def toggle_operations(index,n, x_x, gates, x_qubits,s):
    if x_x[0][index] != 1: #Identical code
            gates += ["x"]
            x_qubits += [n-1-index]
            for x in s:
                x[0][index]= int(not x[0][index])

def conditional_toggle(gates,x_qubits,n,dif,b,s):
    gates += ["cx"]
    sx = [n-1-dif,n-1-b]
    x_qubits += [sx]
    for x in s:
        if x[0][dif] == 1:
            x[0][b] = int(not x[0][b])

def calc_alpha_beta(x_1,x_2):
    beta = x_1[1]
    alpha = x_2[1]

    x_2[1] = alpha+beta
    alpha = alpha/x_2[1]
    beta = beta/x_2[1]

    return alpha, beta

## Quantum Functions
This section details quantum operations essential for the algorithm:

- Quantum functions corresponding to the classical ones, allowing operations on quantum states based on classical pre-processing.

In [4]:
@qfunc
def unitary_control(qubit: QArray[QBit], contrl: QArray[QBit], target: QParam[int]):
      control(lambda: X(qubit[target]), contrl)

@qfunc
def y_rotation(theta: QParam[float], reg: QArray[QBit], target: QParam[int]):
      RY(theta, reg[target])

@qfunc
def my_controlled_unitary(q:QArray[QBit], w:QParam[float], ctrl:QArray[QBit], target:QParam[int]) -> None:
      within_apply(compute=lambda: y_rotation(w,q,target), action=lambda: unitary_control(q, ctrl, target))

@qfunc
def my_unitary(q:QArray[QBit], w:QParam[float],target:QParam[int]) -> None:
      within_apply(compute=lambda: y_rotation(w,q,target), action=lambda: X(q[target]))

## Sparse State Prep
Describes the arguments and setup for preparing a sparse quantum state:

- Initial setup includes defining quantum bit requirements and preparing the environment.
- Integration of classical functions to set up and manipulate quantum state preparation based on given sparse states.

### Algorithm 1
- **Purpose**: The primary function of `algorithm_1` is to efficiently prepare a sparse quantum state based on a given set of sparse state descriptions. It utilizes classical pre-processing to determine the optimal sequence of quantum gates.
- **Process**:
  - It begins by identifying the qubits and the operations that will result in the greatest simplification of the quantum state based on the input data.
  - The algorithm iteratively processes subsets of quantum states, toggling qubits and adjusting their probabilities to converge towards the desired sparse state.
  - Each iteration updates operation lists which are used to guide the quantum operations on the actual quantum hardware.
  

In [5]:
def algorithm_1(s,n, gates, x_qubits, cx_qubits, cg_params, final_state, max_num_ctrls):
    dif_qubits = [] #Where to operate
    dif_values = [] #What operation

    T = s

    dif_qubits, dif_values, t = process_subsets(T, n, dif_qubits, dif_values)
            
    dif = dif_qubits.pop()
    dif_values.pop()

    x_1 = t[0]
    t_prime = [x for x in s if all(x[0][q] == v for q, v in zip(dif_qubits, dif_values))]
    t_prime.remove(x_1)

    dif_qubits, dif_values, t_prime = process_subsets(t_prime, n, dif_qubits, dif_values)        
            
    x_2 = t_prime[0]

    toggle_operations(dif,n,x_1,gates,x_qubits,s)
            
    for b in range(n):
        if b != dif and x_1[0][b] != x_2[0][b]:
            conditional_toggle(gates,cx_qubits,n,dif,b,s)
            
    for b in dif_qubits:
        toggle_operations(b,n,x_2,gates,x_qubits,s)
        

    alpha, beta = calc_alpha_beta(x_1,x_2)
    
    gates += ["cg"]
    cg_param = [alpha,beta, dif_qubits, dif]

    if len(dif_qubits) > 0:
        if len(dif_qubits) >= max_num_ctrls[0]:
            max_num_ctrls[0] = len(dif_qubits)
    else:
        cg_param.remove(dif_qubits)

    cg_params += [cg_param]
    s.remove(x_1)

    if len(s) > 1:
        algorithm_1(s,n, gates, x_qubits, cx_qubits, cg_params, final_state, max_num_ctrls)
    else:
        gates += ["end"]
        final_state += [x_2[0]]

### Main Function Implementation
- **Purpose**: The `main` function orchestrates the overall setup and execution of the sparse state preparation.
- **Functionality**:
  - It initializes the quantum environment with the necessary number of qubits based on the input state descriptions.
  - The sparse states are first converted into a format suitable for quantum operations using `dict_to_3d_array`.
  - The main function then invokes `algorithm_1` to find the necessary operation to prepare the sparse quantum state.
  - After `algorithm_1` completes, the `main` function applies the quantum operations to finalize the state preparation.
  - It is responsible for managing and applying the computed operations to the quantum system, effectively translating the classical pre-processing results into quantum manipulations.

In [6]:
import matplotlib.pyplot as plt 

@qfunc
def main(psi: Output[QArray[QBit]]):

    #Arguments for algorithm1()
    # sparse_states = {'000': 0.25, '001': 0.5, '111': 0.25}
    # sparse_states = {'0001': 0.1, '0011': 0.15, '0111': 0.5, '1111': 0.1, '1011': 0.15} #? (2 ancillas)
    # sparse_states = {'001': 4/168, '100': 64/168, '111': 100/168}
    # sparse_states = {'000011': 0.4, '111111': 0.6}
    # sparse_states = {'0000000101': 0.33, '0000010001': 0.33, '0001100010': 0.34}
    # sparse_states = {'00000000000000000101': 0.6, '00000000000000010001': 0.4}
    global sparse_states 

    NUM_QUBITS = len(next(iter(sparse_states)))

    gates = [] #Stores operations
    x_qubits = []
    cx_qubits = []
    cg_params = []
    final_state = []
    max_num_ctrls = [0]

    #Allocate qubits for output
    allocate(NUM_QUBITS, psi)

    sparse_states = dict_to_3d_array(sparse_states)
    
    if len(sparse_states) > 1:

        algorithm_1(sparse_states, NUM_QUBITS, gates, x_qubits, cx_qubits, cg_params, final_state, max_num_ctrls)

        if max_num_ctrls[0] > 0:
            anc = QArray("anc")
            allocate(max_num_ctrls[0], anc)

        for gate in gates[::-1]:
            if gate == "x":
                # X gate on x_qubits
                X(psi[x_qubits.pop()])
            elif gate == "cx":
                c, t = cx_qubits.pop()
                CX(psi[c], psi[t])
            elif gate == "cg":
                cg = cg_params.pop()
                a = np.sqrt(cg[0] * (cg[0] + cg[1]))
                b = np.sqrt(cg[1] * (cg[0] + cg[1]))

                w = np.arctan2(a, b)

                if len(cg) == 3:
                    # apply G to dif
                    my_unitary(psi, w, NUM_QUBITS - 1 - cg[2])
                else:
                    # apply G to dif controlled on dif_qs
                    for i, d_q in enumerate(cg[2]):
                        CX(psi[NUM_QUBITS - 1 - d_q], anc[i])

                    my_controlled_unitary(psi, w, anc[0:len(cg[2])], NUM_QUBITS - 1 - cg[3])
                    for i, d_q in enumerate(cg[2]):
                        CX(psi[NUM_QUBITS - 1 - d_q], anc[i])

            elif gate == "end":
                for b in range(NUM_QUBITS):
                    if final_state[0][b] == 1:
                        X(psi[NUM_QUBITS-1-b])

    else:
         for b in range(NUM_QUBITS):
            if sparse_states[0][0][b]==1:
                X(psi[NUM_QUBITS-1-b])

for num_qubits in [10, 20]: # n (num qubits) values to test
    # generate all possible n-qubit states
    states = [format(i, '0' + str(num_qubits) + 'b') for i in range(2 ** num_qubits)]
    depths = list()
    ks = range(1, 50+1, 5) # k = number of nonzero coefficients
    for k in ks:
        # Code takes a while to run; print statement to mark progress
        print(f"Running: k = {k}")
        # generate random circuit input based on those parameters 
        state_choices = np.random.choice(states, k, replace=False)
        probabilities = [np.random.random() for i in range(k)] # generate random probabilities 
        probabilities = np.array(probabilities) / sum(probabilities)
        global sparse_states 
        sparse_states = dict()
        for state_choice, probability in zip(state_choices, probabilities):
            sparse_states[state_choice] = probability 
        
        # create model based on sparse state 
        model = create_model(main)
        qprog = synthesize(model)

        circuit = QuantumProgram.from_qprog(qprog)
        depth = circuit.transpiled_circuit.depth
        depths.append(depth)
    plt.yscale("log")
    plt.plot(ks, depths, color="dodgerblue" if num_qubits == 10 else "darkorange", label=("num qubits = " + str(num_qubits)))

# Labels and save plot figure 
plt.title("Increasing Nonzero Coefficients")
plt.xlabel("Number of Nonzero Coefficients (k)")
plt.ylabel("Circuit Depth")
plt.xlim(0, 50)
plt.ylim(0, 10 ** 6)
plt.legend()
plt.savefig("depth_by_k.png")
plt.clf()

#show(qprog)

Running: k = 1


Running: k = 26


/tmp/ipykernel_22907/3210667491.py:39: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  cg_param.remove(dif_qubits)


Running: k = 1
Running: k = 26


/tmp/ipykernel_22907/2284382432.py:105: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  plt.ylim(0, 10 ** 6)


<Figure size 640x480 with 0 Axes>